# LA Crime: Data sources and metadata

#### Load Python tools and Jupyter config

In [79]:
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [5]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [67]:
today = pd.Timestamp("today").strftime("%Y%m%d")
today_epoch = int(pd.Timestamp.today().timestamp())

---

## Metadata
> The LAPD crime data comes from two services: An archive from [2010-2019](https://data.lacity.org/Public-Safety/Crime-Data-from-2010-to-2019/63jg-8b9z/about_data), and a regularly updated listing of incidents from [2020-present](https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data). Each are stored using the Socrata Open Data API (SODA), which allows for bulk download to initiate the project and database model and for regular programatic access to keep the archive updated. 

#### Get metadata for each table

In [71]:
sources = [
    {
        "year": "2010-2019",
        "code": "63jg-8b9z",
        "metadata_url": "https://data.lacity.org/api/views/63jg-8b9z.json",
        "download_url": f"https://data.lacity.org/api/views/63jg-8b9z/rows.csv?fourfour=63jg-8b9z&cacheBust={today_epoch}&date={today}&accessType=DOWNLOAD",
    },
    {
        "year": "2020-present",
        "code": "2nrs-mtv8",
        "metadata_url": "https://data.lacity.org/api/views/2nrs-mtv8.json",
        "download_url": f"https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?fourfour=2nrs-mtv8&cacheBust={today_epoch}&date={today}&accessType=DOWNLOAD",
    },
]

#### Fetch the metadata from both sources to ensure it's identical

In [83]:
dfs = []

for source in sources:
    response = requests.get(source["metadata_url"])
    metadata = pd.DataFrame(response.json()["columns"])[
        [
            "fieldName",
            "name",
            "dataTypeName",
            "description",
        ]
    ]
    dfs.append(metadata)

#### A dataframe listing the column names, types and descriptions

In [84]:
df = pd.concat(dfs).drop_duplicates()

In [85]:
db_schema = df.to_dict("records")

---

## Fetch

#### Function to download the data in bulk to start

In [75]:
def download_crime_data(download_url, file_path):
    response = requests.get(download_url)
    with open(file_path, "wb") as file:
        file.write(response.content)

#### 2010-2019 dataset

In [76]:
download_crime_data(
    "https://data.lacity.org/api/views/63jg-8b9z/rows.csv?fourfour=63jg-8b9z&cacheBust=1712241918&date=20240404&accessType=DOWNLOAD",
    "../data/raw/crime_data_2010_2019.csv",
)

#### 2020-present dataset

In [ ]:
download_crime_data(
    "https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?fourfour=2nrs-mtv8&cacheBust=1712241918&date=20240404&accessType=DOWNLOAD",
    "../data/raw/crime_data_2020_present.csv",
)

---

#### Export metadata

In [86]:
df.to_csv("../data/raw/crime_data_metadata.csv", index=False)